# **Facephoto_to_periocular**

```
Facephoto (/images) → RT-DETRで眼検出 → /labelsに保存
→ labelsに従って切り抜き。左右25%ずつ広く取る、縦は横幅と一緒にする。端からはみ出る場合には黒色でletterboxに
→ 右眼は_R、左眼は_Lを追加して保存
```

In [1]:
orig_img_dir = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295\images"
orig_label_dir = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295\labels"
dst_img_dir = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\images"

In [2]:
import os
import cv2
import numpy as np
import glob
import shutil
from tqdm import tqdm

# 1. 元の画像とラベルファイルを読み込み
# 2. ラベルファイルから眼の位置情報を取得
# 3. 眼の周辺を左右に25%ずつ拡張して切り抜き
# 4. 右眼は_R、左眼は_Lを付けて保存
# 5. 画像からはみ出る場合は黒色でパディング

# ディレクトリ定義
orig_img_dir = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295\images"
orig_label_dir = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295\labels"
dst_img_dir = r"C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\images"

# 保存先ディレクトリが存在する場合は削除して作り直す
if os.path.exists(dst_img_dir):
    print(f"既存のディレクトリを削除します: {dst_img_dir}")
    shutil.rmtree(dst_img_dir)
os.makedirs(dst_img_dir, exist_ok=True)
print(f"新しい出力ディレクトリを作成しました: {dst_img_dir}")

# 対応する画像拡張子の定義
img_extensions = ['*.jpg', '*.JPG', '*.jpeg', '*.JPEG', '*.png', '*.PNG', '*.tif', '*.TIF', '*.tiff', '*.TIFF']

# すべての画像ファイルを検索
all_images = []
for ext in img_extensions:
    all_images.extend(glob.glob(os.path.join(orig_img_dir, ext)))

print(f"合計 {len(all_images)} 枚の画像が見つかりました")

# 各画像を処理
for img_path in tqdm(all_images, desc="画像の処理"):
    # 拡張子なしのベースネームを取得
    img_basename = os.path.splitext(os.path.basename(img_path))[0]
    
    # 対応するラベルファイルを探す
    label_path = os.path.join(orig_label_dir, f"{img_basename}.txt")
    
    # ラベルファイルが存在しない場合はスキップ
    if not os.path.exists(label_path):
        tqdm.write(f"警告: {img_basename} のラベルファイルが見つかりません。スキップします。")
        continue
    
    # 画像を読み込む
    original_image = cv2.imread(img_path)
    if original_image is None:
        tqdm.write(f"警告: 画像 {img_path} の読み込みに失敗しました。スキップします。")
        continue
    
    img_height, img_width = original_image.shape[:2]
    
    # アノテーションを読み込む
    with open(label_path, 'r') as f:
        lines = f.readlines()
    
    # 各アノテーション（眼）を処理
    for line in lines:
        parts = line.strip().split()
        if len(parts) < 5:
            continue
        
        cls_id = int(parts[0])
        x_center = float(parts[1])
        y_center = float(parts[2])
        width = float(parts[3])
        height = float(parts[4])
        
        # 右眼(R)か左眼(L)かを判定
        class_label = "R" if cls_id == 0 else "L"
        
        # 正規化された座標をピクセル値に変換
        x_center_px = x_center * img_width
        y_center_px = y_center * img_height
        width_px = width * img_width
        height_px = height * img_height
        
        # 拡張された寸法を計算（左右25%ずつ）
        expanded_width = width_px * 1.5  # 元のサイズ + 50%（左右それぞれ25%）
        expanded_height = expanded_width  # 正方形にする
        
        # 新しい切り取り座標を計算
        new_x1 = int(x_center_px - expanded_width / 2)
        new_y1 = int(y_center_px - expanded_height / 2)
        new_x2 = int(x_center_px + expanded_width / 2)
        new_y2 = int(y_center_px + expanded_height / 2)
        
        # 必要な場合のパディングを計算
        pad_left = max(0, -new_x1)
        pad_top = max(0, -new_y1)
        pad_right = max(0, new_x2 - img_width)
        pad_bottom = max(0, new_y2 - img_height)
        
        # 画像の境界内の実際の切り取り座標を計算
        crop_x1 = max(0, new_x1)
        crop_y1 = max(0, new_y1)
        crop_x2 = min(img_width, new_x2)
        crop_y2 = min(img_height, new_y2)
        
        # 領域を切り取る
        cropped = original_image[crop_y1:crop_y2, crop_x1:crop_x2].copy()
        
        # 必要な場合はパディングを適用（画像がはみ出る場合）
        if pad_left > 0 or pad_top > 0 or pad_right > 0 or pad_bottom > 0:
            padded_width = crop_x2 - crop_x1 + pad_left + pad_right
            padded_height = crop_y2 - crop_y1 + pad_top + pad_bottom
            padded = np.zeros((padded_height, padded_width, 3), dtype=np.uint8)  # 黒背景
            padded[pad_top:pad_top+(crop_y2-crop_y1), pad_left:pad_left+(crop_x2-crop_x1)] = cropped
            cropped = padded
        
        # 出力ファイル名を作成
        output_filename = f"{img_basename}_{class_label}.jpg"
        output_path = os.path.join(dst_img_dir, output_filename)
        
        # 切り取った画像を保存
        cv2.imwrite(output_path, cropped)

print("処理が完了しました！")

新しい出力ディレクトリを作成しました: C:\Users\CorneAI\YOLOv11_Mobius\eyelid_image\1-295_periocular\images
合計 4640 枚の画像が見つかりました


画像の処理:   2%|▏         | 92/4640 [00:08<05:57, 12.74it/s]     

警告: 115-20030213-54-155844_1132ce9beb104261c06f0c9b780a352d40cd74807477d43da38643ebc48cb4fc のラベルファイルが見つかりません。スキップします。


画像の処理:   6%|▌         | 257/4640 [00:24<05:30, 13.26it/s]     

警告: 138-20140903-11-090100_ac43668da8f46190a60a8ca7da88107187b4a02d94fbe6bb1b7800a9add12bef のラベルファイルが見つかりません。スキップします。


...

警告: 80-20001122-22-131410_cf25ca912f676b11bdb6f9cfe75ec17410862b9599fa48a36fede69cf3697523 のラベルファイルが見つかりません。スキップします。
警告: 80-20010731-22-122059_63f1e91352399faa423a4ef50ded40f5749d62b92e914a730937365bc0400ea3 のラベルファイルが見つかりません。スキップします。


画像の処理:  98%|█████████▊| 4568/4640 [05:29<00:06, 11.42it/s]     

警告: 87-20000706-71-130236_e37f4b5bafbf692128e87bdccd6c51bcea1ed844a2c3591469c28b1008c6bd2b のラベルファイルが見つかりません。スキップします。
警告: 87-20000921-71-123550_97a549a3bab277a305ac4ceebb8c57d6ca073fc983e2e2938e5b34ad019fad5a のラベルファイルが見つかりません。スキップします。


画像の処理: 100%|██████████| 4640/4640 [05:35<00:00, 13.82it/s]     

警告: 128-20190508-47-134032_5db81163ba9370f7b47e896a86662b51eecc24df7b5e095a28e728cb3d823219 のラベルファイルが見つかりません。スキップします。
警告: 128-20190508-47-135005_fed5582711f2ebfae749ae7bc8ba6a1dfe8613ffc26cb4d585e4cb745c263c8b のラベルファイルが見つかりません。スキップします。
警告: 128-20190508-47-134032_5db81163ba9370f7b47e896a86662b51eecc24df7b5e095a28e728cb3d823219 のラベルファイルが見つかりません。スキップします。
警告: 128-20190508-47-135005_fed5582711f2ebfae749ae7bc8ba6a1dfe8613ffc26cb4d585e4cb745c263c8b のラベルファイルが見つかりません。スキップします。
処理が完了しました！
